# Obtaining the data from datasources

The first part of this project is to obtain the data from the different datsources we have chosen. Also, there are some datasources that for purposes of this project, we have created ourselves. In this notebook we will explain how the data is created, if needed, and how the data is collected from the different data sources. Specifically, we will obtain the data from three different data sources:
1. [huggingface dataset](https://huggingface.co/datasets/Moaaz55/skin_cancer_questions_answers). This dataset is used for **joel pots explicar aqui**
2. Self made audio dataset. This dataset is used for ...
3. Wikipedia web scrapping information. This dataset is used for...

In the next sections, we will discuss how the data is obtained to further insert them into the first zone of the pipeline.

## Huggingface Dataset

Explicar huggingface dataset

In [1]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_json("hf://datasets/Moaaz55/skin_cancer_questions_answers/dataset.json", lines=True)

def limpiar_dataset(df, columna='Answer'):
    # Convertir a string y limpiar
    df_temp = df.copy()
    df_temp[columna] = df_temp[columna].astype(str)
    
    # filter valid answers.
    df_limpio = df_temp[
        df_temp[columna].notna() &
        (df_temp[columna].str.strip() != '') &
        (df_temp[columna].str.strip() != 'nan') &
        (df_temp[columna].str.strip() != 'None') &
        (df_temp[columna].str.strip() != 'null') &
        (df_temp[columna].str.len() > 10)  # Mínimo 10 caracteres
    ]
    
    print(f" Limpieza completada:")
    print(f"  Original: {len(df)} filas")
    print(f"  Limpio: {len(df_limpio)} filas")
    print(f"  Eliminadas: {len(df) - len(df_limpio)} filas")
    
    return df_limpio


df = limpiar_dataset(df)
df = df.sample(n=100, random_state=42)
df_text = df
df_text = df.apply(lambda row: f"Q: {row['Question']}\nA: {row['Answer']}\n", axis=1)
df_text
with open("preguntas_respuestas.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(df_text.tolist()))

/home/oriol/Documentos/MDS/ADSDB/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 Limpieza completada:
  Original: 798 filas
  Limpio: 757 filas
  Eliminadas: 41 filas
